In [ ]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")

In [1]:
from tensorflow import keras
import tensorflow as tf

2025-03-10 13:22:16.186925: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 13:22:16.369565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741612936.455446     510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741612936.480051     510 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 13:22:16.684902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from Pills.GenerativeSequence import GenerativeSequence
from PillNet import PillNet

ModuleNotFoundError: No module named 'Pills'

In [ ]:
SOURCE_PATH = '/workspaces/dev/datasets/pills/data'
LABEL_PATH = '/workspaces/dev/datasets/pills/class_label.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data'

In [ ]:
TRAIN_DATA_SIZE = 60000
VALIDATION_DATA_SIZE = 20000
BATCH_SIZE = 4
INPUT_SIZE = (256, 256, 3)

In [ ]:
model = PillNet(BATCH_SIZE, 50, INPUT_SIZE)
model.summary()

I0000 00:00:1741604695.499413   84791 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9502 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "pill_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ROI (Sequential)                │ (None, 64, 64, 1)      │       138,337 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DetectionHead (Sequential)      │ (None, 4, 4, 4)        │       675,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FeatureExtractor (Sequential)   │ (None, 64, 64, 64)     │        32,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CentroidHead (Sequential)       │ (None, 2)              │         5,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SegmentationHead (Sequential)   │ (None, 64, 64, 1)      │        34,385 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ClassificationHead (Sequential) │ (None, 50)             │       210,728 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,096,396 (4.18 MB)

 Trainable params: 1,096,396 (4.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_sequence = GenerativeSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)
validation_sequence = GenerativeSequence(
  VALIDATION_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2]), dtype=tf.float32),
    {
      "roi": tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1]), dtype=tf.bool),
      "centroid": tf.TensorSpec(shape=(BATCH_SIZE * 16, 2), dtype=tf.float32),
      "detection": tf.TensorSpec(shape=(BATCH_SIZE * 16, 4), dtype=tf.float32),
      "segmentation": tf.TensorSpec(shape=(BATCH_SIZE * 16, INPUT_SIZE[0]//4, INPUT_SIZE[1]//4), dtype=tf.bool),
      "classification": tf.TensorSpec(shape=(BATCH_SIZE * 16), dtype=tf.uint32)
    }
  )
)

In [ ]:
model.compile()

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath="/workspaces/dev/models/R-CNN/checkpoints/pill_net.keras",
  monitor="val_loss",
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [ ]:
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=100,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

Epoch 1/100


I0000 00:00:1741604737.017217   84912 service.cc:148] XLA service 0x7fe70c002cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741604737.017255   84912 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2025-03-10 11:05:37.151996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1741604737.350518   84912 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert/Assert
W0000 00:00:1741604737.350612   84912 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_1/Assert
W0000 00:00:1741604737.351622   84912 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_2/Assert
W0000 00:00:1741604737.351687   84912 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while

    1/15000 ━━━━━━━━━━━━━━━━━━━━ 63:36:15 15s/step - centroid_loss: 0.1519 - classification_loss: 3.9296 - detection_loss: 0.0402 - roi_loss: 0.7044 - segmentation_loss: 0.6965 - total_loss: 5.5227

I0000 00:00:1741604746.834035   84912 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1741604747.210224   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert/Assert
W0000 00:00:1741604747.210275   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_1/Assert
W0000 00:00:1741604747.210375   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_2/Assert
W0000 00:00:1741604747.210380   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_3/Assert
W0000 00:00:1741604747.210394   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_4/Assert
W0000 00:00:1741604747.210399   84909 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_5/Assert
W0000 00:00:174160

 1601/15000 ━━━━━━━━━━━━━━━━━━━━ 18:23:25 5s/step - centroid_loss: 0.0041 - classification_loss: 3.9127 - detection_loss: 0.0016 - roi_loss: 0.1741 - segmentation_loss: 0.3567 - total_loss: 4.4492